In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import sys
import pickle

path = os.getcwd().split(os.sep + 'GUI')[0]
if path not in sys.path:
    print("not here")
    sys.path.append(path)

from neurolib.models.aln import ALNModel
from neurolib.utils import plotFunctions as plotFunc
from neurolib.utils import costFunctions as cost
import neurolib.dashboard.functions as functions
import neurolib.dashboard.data as data
    
# This will reload all imports as soon as the code changes
%load_ext autoreload
%autoreload 2 

not here


In [2]:
aln = ALNModel()
N = aln.params.N

data.set_parameters(aln)

state_vars = aln.state_vars
init_vars = aln.init_vars

##############################################################
def setinit(init_vars_, model):
    state_vars = model.state_vars
    init_vars = model.init_vars
    for iv in range(len(init_vars)):
        for sv in range(len(state_vars)):
            if state_vars[sv] in init_vars[iv]:
                #print("set init vars ", )
                if model.params[init_vars[iv]].ndim == 2:
                    model.params[init_vars[iv]][0,:] = init_vars_[sv]
                else:
                    model.params[init_vars[iv]][0] = init_vars_[sv]
                    
##############################################################               
def setmaxmincontrol(max_c_c, min_c_c, max_c_r, min_c_r):
    import numpy as np
    
    max_cntrl = np.zeros(( 6 ))
    min_cntrl = np.zeros(( 6 ))
    
    max_cntrl[0] = max_c_c
    min_cntrl[0] = min_c_c
    max_cntrl[1] = max_c_c
    min_cntrl[1] = min_c_c
    max_cntrl[2] = max_c_r
    min_cntrl[2] = min_c_r
    max_cntrl[3] = max_c_r
    min_cntrl[3] = min_c_r
    max_cntrl[4] = max_c_r
    min_cntrl[4] = min_c_r
    max_cntrl[5] = max_c_r
    min_cntrl[5] = min_c_r
            
    return max_cntrl, min_cntrl

In [3]:
##### LOAD BOUNDARIES
data_file = 'bi.pickle'
with open(data_file,'rb') as f:
    load_array= pickle.load(f)
exc = load_array[0]
inh = load_array[1]
print(len(exc))

147


In [18]:
t0_array = np.arange(380., -1, -1.)
t_pen = t0_array / 400.

bestControl_1 = [None] * len(t_pen)
bestState_1 = [None] * len(t_pen)
cost_1 = [None] * len(t_pen)
runtime_1 = [None] * len(t_pen)
grad_1 = [None] * len(t_pen)
phi_1 = [None] * len(t_pen)
costnode_1 = [None] * len(t_pen)
weights_1 = [None] * len(t_pen)

In [5]:
initVars = [None] * len(exc)
target = [None] * len(exc)
cost_uncontrolled = [None] * len(exc)

cgv_list = [None, "HS", "FR", "PR", "CD", "LS", "DY", "WYL", "HZ", None]

In [15]:
dur_pre = 10
dur_post = 10

i = 50

n_pre = int(np.around(dur_pre/aln.params.dt + 1.,1))
n_post = int(np.around(dur_post/aln.params.dt + 1.,1))

tol = 1e-32
start_step = 10.
c_scheme = np.zeros(( 1,1 ))
c_scheme[0,0] = 1.
u_mat = np.identity(1)
u_scheme = np.array([[1.]])

c_var = [ [0], [1], [0,1]]
p_var = [ [0], [0], [0]]

### CURRENTS
cntrl_vars_0 = [0,1]
prec_vars = [0]

max_I = [-3., 3.]
factor_wp = 1. / 0.05
factor_ws = 0.
factor_we = 1.
dur = 400
trans_time = 0.95
    
maxC = [5., -5., 0.18, 0.]

n_dur = int(np.around(dur/aln.params.dt + 1.,1))
max_cntrl, min_cntrl = setmaxmincontrol(maxC[0], maxC[1], maxC[2], maxC[3])

In [16]:
init_file = 'control_4.pickle'
final_file_1 = 'control_tradeoff_4b.pickle'

In [19]:
if os.path.isfile(init_file) :
    print("init file found")
    
    with open(init_file,'rb') as f:
        load_array = pickle.load(f)

    print(len(load_array[0]))

    bestControl_1[0] = load_array[0][i]
    bestState_1[0] = load_array[1][i]
    cost_1 [0]= load_array[2][i]
    runtime_1[0] = load_array[3][i]
    grad_1 [0]= load_array[4][i]
    phi_1[0] = load_array[5][i]
    costnode_1[0] = load_array[6][i]
    weights_1[0] = load_array[7][i]

init file found
147


In [10]:
if os.path.isfile(final_file_1) :
    print("final file found")
    
    with open(final_file_1,'rb') as f:
        load_array = pickle.load(f)

    print(len(load_array[0]))

    bestControl_1 = load_array[0]
    bestState_1 = load_array[1]
    cost_1 = load_array[2]
    runtime_1 = load_array[3]
    grad_1 = load_array[4]
    phi_1 = load_array[5]
    costnode_1 = load_array[6]
    weights_1 = load_array[7]

In [21]:
# get initial parameters and target states

print("------- ", i, exc[i], inh[i])
aln.params.ext_exc_current = exc[i] * 5.
aln.params.ext_inh_current = inh[i] * 5.

aln.params.duration = 3000.

control0 = aln.getZeroControl()
control0 = functions.step_control(aln, maxI_ = max_I[0])

aln.run(control=control0)

target_rates = np.zeros((2))
target_rates[0] = aln.rates_exc[0,-1] 
target_rates[1] = aln.rates_inh[0,-1]

control0 = functions.step_control(aln, maxI_ = max_I[1])
aln.run(control=control0)

init_state_vars = np.zeros(( len(state_vars) ))
for j in range(len(state_vars)):
    if aln.state[state_vars[j]].size == 1:
        init_state_vars[j] = aln.state[state_vars[j]][0]
    else:
        init_state_vars[j] = aln.state[state_vars[j]][0,-1]

initVars[i] = init_state_vars

aln.params.duration = dur

target[i] = aln.getZeroTarget()
target[i][:,0,:] = target_rates[0]
target[i][:,1,:] = target_rates[1]

-------  50 0.47500000000000014 0.6000000000000003


In [20]:
# get uncontrolled cost

data.set_parameters(aln)
aln.params.ext_exc_current = exc[i] * 5.
aln.params.ext_inh_current = inh[i] * 5.
aln.params.duration = dur

cgv = None

In [22]:
print(np.argmax(np.abs(bestControl_1[0]), axis=2))
print(bestControl_1[0][0,0,1650:1700]/5.)
print(bestControl_1[0][0,0,1674]/5.)

max_percent = (np.argmax(np.abs(bestControl_1[0]), axis=2)[0,0] -100) / 3800
print(max_percent)

[[1514 1494    0    0    0    0]]
[-0.05751683 -0.05697911 -0.05644384 -0.0559128  -0.05538419 -0.05485816
 -0.05433471 -0.05381599 -0.05330156 -0.05278987 -0.05228085 -0.05177645
 -0.05127469 -0.05078954 -0.05030694 -0.04986323 -0.04942393 -0.04898738
 -0.04855358 -0.04812249 -0.04769409 -0.04726827 -0.04684495 -0.04642402
 -0.04600545 -0.04558917 -0.04517509 -0.04476315 -0.04435334 -0.04394566
 -0.04354011 -0.04313669 -0.04273688 -0.04233917 -0.04194427 -0.04155249
 -0.04116505 -0.0407798  -0.04039676 -0.04001702 -0.0396399  -0.03926505
 -0.03889238 -0.03852188 -0.03815363 -0.03778886 -0.0374262  -0.03706575
 -0.03670746 -0.03635133]
-0.04600545478493252
0.3721052631578947


In [23]:
factor_iteration = 1.

##### initial guess
weight_ = 1.
cost.setParams(weight_ * factor_wp, weight_ * factor_we, weight_ * factor_ws)

setinit(initVars[i], aln)

weights_1[0] = cost.getParams()

for j in range(1, 381):

    dur = 400. - np.around(j/10., 1)
    T = 4001 - j
    aln.params.duration = dur

    factor_wp = dur / 20.
    cost.setParams(weight_ * factor_wp, weight_ * factor_we, weight_ * factor_ws)

    print('-----', j)
    setinit(initVars[i], aln)

    max_it = 3.
    if j > 170:
        max_it = 40.
    elif j > 140:
        max_it = 30.
    elif j > 100.:
        max_it = 20.
    
    if type(bestControl_1[j]) == type(None):
        total_shift = round(j * max_percent * 10)
        prev_shift = round( (j-1) * max_percent * 10)
        shift_ind = total_shift - prev_shift
        #print(total_shift, prev_shift, shift_ind)
        control0 = np.zeros(( bestControl_1[j-1][:,:,100:-101].shape ))
        control0[:,:,:-shift_ind] = 1. * bestControl_1[j-1][:,:,100+shift_ind:-101]
    else:
        if bestControl_1[j].shape[2] == T+200:
            control0 = bestControl_1[j][:,:,100:-100]
        else:
            control0 = bestControl_1[j][:,:,100:-(100+j)]

    bestControl_1[j], bestState_1[j], cost_1[j], runtime_1[j], grad_1[j], phi_1[j], costnode_1[j] = aln.A1(
        control0, target[i], c_scheme, u_mat, u_scheme, max_iteration_ = max_it, tolerance_ = tol,
        startStep_ = start_step, max_control_ = max_cntrl, min_control_ = min_cntrl, t_sim_ = dur,
        t_sim_pre_ = dur_pre, t_sim_post_ = dur_post, CGVar = cgv, control_variables_ = [0,1],
        prec_variables_ = prec_vars, transition_time_ = t_pen[j])

    print(costnode_1[j][0][0][0], costnode_1[j][2][0][1])

    print(costnode_1[0][0][0][0], costnode_1[0][2][0][1])
        
    with open(final_file_1,'wb') as f:
        pickle.dump([bestControl_1, bestState_1, cost_1, runtime_1, grad_1, phi_1,
                costnode_1, weights_1], f)

set cost params:  20.0 1.0 0.0
set cost params:  19.994999999999997 1.0 0.0
----- 1
interpolate adjoint :  True True True
RUN  0 , total integrated cost =  3.5211823133352693
Gradient descend method:  None
RUN  1 , total integrated cost =  3.5211822180916412


ERROR:root:Problem in initial value trasfer


RUN  2 , total integrated cost =  3.5211822180842813
RUN  3 , total integrated cost =  3.5211822180672296
RUN  3 , total integrated cost =  3.5211822180672296
Improved over  3  iterations in  0.8400544570031343  seconds by  2.7055696421030007e-06  percent.
Problem in initial value trasfer:  Vmean_exc -73.82498121728527 -73.82822914900795
9.162601673857172e-10 0.12950548299152465
5.551237769899523e-09 0.12950540574463307
set cost params:  19.990000000000002 1.0 0.0
----- 2
interpolate adjoint :  True True True
RUN  0 , total integrated cost =  3.5211822207469057
Gradient descend method:  None
RUN  1 , total integrated cost =  3.5211822205612893


ERROR:root:Problem in initial value trasfer


RUN  2 , total integrated cost =  3.5211822205510264
RUN  3 , total integrated cost =  3.5211822205368146
RUN  3 , total integrated cost =  3.5211822205368146
Improved over  3  iterations in  0.6417241830058629  seconds by  5.966498406451137e-09  percent.
Problem in initial value trasfer:  Vmean_exc -73.84879309327464 -73.85193060691489
9.481127369081806e-10 0.12950549717957846
5.551237769899523e-09 0.12950540574463307
set cost params:  19.985 1.0 0.0
----- 3
interpolate adjoint :  True True True
RUN  0 , total integrated cost =  3.5211822233187706
Gradient descend method:  None
RUN  1 , total integrated cost =  3.521182223150717
RUN  2 , total integrated cost =  3.5211807523852987


...

RUN  40 , total integrated cost =  4.499635502467152
RUN  40 , total integrated cost =  4.499635502467152
Improved over  40  iterations in  5.222680784005206  seconds by  2.0061710504606083  percent.
Problem in initial value trasfer:  Vmean_exc -74.52687254873602 -74.52687255943269
0.004981749200846262 0.14583028567664189
5.551237769899523e-09 0.12950540574463307
set cost params:  18.29 1.0 0.0
----- 342
interpolate adjoint :  True True True
RUN  0 , total integrated cost =  4.655861649383522
Gradient descend method:  None
RUN  1 , total integrated cost =  4.576864274604354
RUN  2 , total integrated cost =  4.57636746639396
RUN  3 , total integrated cost =  4.575850447691166
RUN  4 , total integrated cost =  4.575472768235357
RUN  5 , total integrated cost =  4.5749188405190395
RUN  6 , total integrated cost =  4.574465412253066
RUN  7 , total integrated cost =  4.573922058876082
RUN  8 , total integrated cost =  4.57345857829421
RUN  9 , total integrated cost =  4.572834163206675
RUN 

ERROR:root:Problem in initial value trasfer


RUN  40 , total integrated cost =  4.564174277706577
RUN  40 , total integrated cost =  4.564174277706577
Improved over  40  iterations in  5.019464297001832  seconds by  1.969289007741125  percent.
Problem in initial value trasfer:  Vmean_exc -74.52687262237563 -74.52687263273225
0.005353093793254117 0.14698996829848143
5.551237769899523e-09 0.12950540574463307
set cost params:  18.285 1.0 0.0
----- 343
interpolate adjoint :  True True True
RUN  0 , total integrated cost =  4.706216938371231
Gradient descend method:  None
RUN  1 , total integrated cost =  4.6447863849688416
RUN  2 , total integrated cost =  4.635477111151822
RUN  3 , total integrated cost =  4.634010948915311
RUN  4 , total integrated cost =  4.633955004283326
RUN  5 , total integrated cost =  4.633915230211893
RUN  6 , total integrated cost =  4.6338835402416665
RUN  7 , total integrated cost =  4.633855204553718
RUN  8 , total integrated cost =  4.633831946830458
RUN  9 , total integrated cost =  4.633797126129897
R

ERROR:root:Problem in initial value trasfer


Control only changes marginally.
RUN  31 , total integrated cost =  4.63318536401865
Improved over  31  iterations in  4.20904578200134  seconds by  1.5518106221821597  percent.
Problem in initial value trasfer:  Vmean_exc -74.52687269486631 -74.5268727048882
0.005738362458457602 0.14826879852638358
5.551237769899523e-09 0.12950540574463307
set cost params:  18.28 1.0 0.0
----- 344
interpolate adjoint :  True True True
RUN  0 , total integrated cost =  4.80741480245045
Gradient descend method:  None
RUN  1 , total integrated cost =  4.720506865255844
RUN  2 , total integrated cost =  4.7198327082437554
RUN  3 , total integrated cost =  4.719291524966182
RUN  4 , total integrated cost =  4.718369270897073
RUN  5 , total integrated cost =  4.717522480361373
RUN  6 , total integrated cost =  4.715165609889135
RUN  7 , total integrated cost =  4.7136596376200055
RUN  8 , total integrated cost =  4.711962828723694
RUN  9 , total integrated cost =  4.710752256744907
RUN  10 , total integrate